# Importazione librerie e funzioni

In [1]:
import string
import sys
import io 
import nltk
import pandas as pd
import numpy as np

# Stopwords
from gensim.parsing.preprocessing import remove_stopwords
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('italian'))
nltk_stopwords = nltk.corpus.stopwords.words('italian')
# Valutare di aggiungere eventuali forme arcariche delle stopwords italiane, o se c'è necessità di estendere questa lista
  
# Divido il testo in frasi in base ai punti
nltk.download('punkt')

# Tokenizer
from nltk.tokenize import word_tokenize 

# Per il discorso lemmatizzazione dobbiamo valutare come muoverci con l'italiano

# Lemmatization
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Contatore parole uniche
from collections import Counter

# Per esplorare risultati
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cafe6\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Preprocessing (NO lemmatization)
def preprocessing(file_name):

  output=""
  with open(file_name, encoding='utf-8') as f:
      for line in f:
          if not line.isspace(): # Rimuovo linee vuote
              output+=line

  # Divido il testo in frasi, basandomi sui punti
  output_sentences = nltk.tokenize.sent_tokenize(output)

  # Valutare se è necessario eliminare delle righe all'inizio o alla fine dei .txt se presentano licenze, ...
  # output_sentences = output_sentences[:-]

  filtered_sentences = []
  # 'Pulisco' ogni frasi, una alla volta
  for sentence in output_sentences:
    # Metto tutto in lower case
    lower_sentence=sentence.lower()
    # Rimuovo caratteri non alfa numerici
    noalfa_sentence = [w for w in word_tokenize(lower_sentence) if (w.isalpha()==True)]
    # Rimuovo le stopwords e le parole di un solo carattere che potrebbero non essere incluse nella lista delle stopwords
    filtered_sentence = [w for w in noalfa_sentence if ((w not in stops) and (len(w) > 1))]
    # Ricostruisco la lista con le frasi 'pulite'
    if filtered_sentence:
      filtered_sentences.append(filtered_sentence)

  return filtered_sentences

In [3]:
import os
from gensim.models import Word2Vec
from cade.cade import CADE

In [4]:
# Funzione per addestrare n modelli per corpus

def training_W2V(sentences, text, n_mod): #file con frasi del corpus, nome del corpus, numero di modelli da addestrare

  for k in range(n_mod):
    model = Word2Vec(sentences = sentences,
                    #window = 5, default value
                    min_count=10, #not consider word with absolute frequency <10 
                    size=300, #vector size 
                    sg = 1, #skipgram algorithm
                    hs = 0,
                    negative = 5, #negative sampling with 5 noise words
                    workers = 5, #faster process
                    iter = 6 #6 iterations
                    )
  
    model.save(text.lower() + "_" + str(k) + ".model")


# Funzione per addestrare n slice per corpus con CADE

# !cat corpus1.txt corpus2.txt corpus3.txt ... > compass.txt

def training_CADE(texts, n_mod): #lista contenente i corpus usati per creare la compass in ordine, numero di slices da addestrare
                                 #la lista dev'essere composta dal nome esatto del file txt ma senza l'estensione

  aligner = CADE(min_count=10,  
                  size=300,
                  sg = 1, 
                  #hs = 0,
                  ns = 5, 
                  workers = 5,
                  siter = 6)

  for k in range(n_mod):
    aligner.train_compass('compass.txt', overwrite=True)
    for text in texts:
      model_slice = aligner.train_slice(text + ".txt") #per trainare le slice ho bisogno del nome esatto del file .txt con cui ho creato il compasso
      model_slice.save(text + '_cade_' + str(k) + '.model') #qui posso salvare il modello con un nome a piacere

In [6]:
import string

def simple_preproc(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# Prova training w2v

In [54]:
os.chdir('C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data')
path_periodi_letterari = 'C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Preprocessing/fasi_letterarie/Frasi NON lemmatizzate/'
path_periodi_storici = 'C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Preprocessing/periodi_storici/'

In [10]:
with open(path_periodi_letterari + '7_illuminismo_neoclassicismo.txt', encoding='utf-8') as read_file:
    illuminismo_neoclassicismo_sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]

In [19]:
illuminismo_neoclassicismo_model = Word2Vec(sentences = illuminismo_neoclassicismo_sentences,
                                            #window = 5, default value
                                            min_count=10, #not consider word with absolute frequency <10
                                            vector_size=300, #vector size
                                            sg = 1, #skipgram algorithm
                                            hs = 0,
                                            negative = 5, #negative sampling with 5 noise words
                                            workers = 5, #faster process
                                            epochs = 6 #6 iterations
                                           )

# circa 1 minuto per il training (file 14MB)

In [43]:
illuminismo_neoclassicismo_model.wv.most_similar('donna')

[('sposa', 0.7747417688369751),
 ('madre', 0.7651275396347046),
 ('amata', 0.7642540335655212),
 ('unica', 0.7609820365905762),
 ('sposo', 0.7592434287071228),
 ('ottima', 0.7581976652145386),
 ('egregia', 0.7551229596138),
 ('virtuosa', 0.7550404667854309),
 ('accorta', 0.7507830262184143),
 ('avvenente', 0.7493323087692261)]

In [45]:
with open(path_periodi_letterari + '8_romanticismo.txt', encoding='utf-8') as read_file:
    romanticismo_sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]

In [46]:
romanticismo_model = Word2Vec(sentences = romanticismo_sentences,
                                            #window = 5, default value
                                            min_count=10, #not consider word with absolute frequency <10
                                            vector_size=300, #vector size
                                            sg = 1, #skipgram algorithm
                                            hs = 0,
                                            negative = 5, #negative sampling with 5 noise words
                                            workers = 5, #faster process
                                            epochs = 6 #6 iterations
                                           )

# circa 3 minuti per il training (file 45MB)

In [53]:
romanticismo_model.wv.most_similar('donna')

[('dama', 0.5912120938301086),
 ('prassede', 0.5745195150375366),
 ('ragazza', 0.5625020861625671),
 ('costei', 0.5556043982505798),
 ('fanciulla', 0.5509609580039978),
 ('marito', 0.5431538224220276),
 ('femmina', 0.5390230417251587),
 ('vedova', 0.5338456034660339),
 ('pudica', 0.532570481300354),
 ('orfana', 0.530032753944397)]

In [55]:
with open(path_periodi_storici + '1814_1860_Risorgimento.txt', encoding='utf-8') as read_file:
    risorgimento_sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]

In [56]:
risorgimento_model = Word2Vec(sentences = risorgimento_sentences,
                                            #window = 5, default value
                                            min_count=10, #not consider word with absolute frequency <10
                                            vector_size=300, #vector size
                                            sg = 1, #skipgram algorithm
                                            hs = 0,
                                            negative = 5, #negative sampling with 5 noise words
                                            workers = 5, #faster process
                                            epochs = 6 #6 iterations
                                           )

# circa 5 minuti per il training (file 83MB)

In [60]:
risorgimento_model.wv.most_similar('donna')

[('paola', 0.5707671642303467),
 ('fanciulla', 0.5651019811630249),
 ('donzella', 0.5281831622123718),
 ('gentildonna', 0.5272212624549866),
 ('clelia', 0.5271718502044678),
 ('colei', 0.5255793333053589),
 ('femmina', 0.5114826560020447),
 ('costei', 0.507268488407135),
 ('piccarda', 0.4998212158679962),
 ('prassede', 0.4954227805137634)]

# Ricostruzione corpus per CADE e prova training

In [74]:
files = [f for f in os.listdir(path_periodi_letterari)]
print(files)

['10_decadentismo.txt', '11_avanguardie_primo_novecento.txt', '12_neorealismo.txt', '1_duecento_poetica_e_prosa Medievo.txt', '2_dolce_stil_novo.txt', '3_petrarchismo.txt', '4_boccaccio_e_umanesimo.txt', '5_manierismo_e_barocco.txt', '6_classicismo_arcadia.txt', '7_illuminismo_neoclassicismo.txt', '8_romanticismo.txt', '9_verismo.txt']


In [68]:
path_cade_periodi_letterari = 'C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Preprocessing/fasi_letterarie/Corpus CADE/'

In [70]:
for file in files:
    with open(path_periodi_letterari + file, encoding='utf-8') as read_file:
        sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]
        with open(path_cade_periodi_letterari+'CADE_'+file, mode='w', encoding='utf-8') as f:
            for s in sentences:
                for w in s:
                    f.write(w+" ")
                f.write('\n')

In [79]:
path_autori = 'C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Preprocessing/autori/frasi_NON_lemmatizzate/'
files_autori = [f for f in os.listdir(path_autori)]
print(files_autori)

['Alessandro Manzoni.txt', 'Dante Alighieri.txt', 'Dino Buzzati.txt', 'Francesco Petrarca.txt', "Gabriele D'Annunzio.txt", 'Giacomo Leopardi.txt', 'Giovanni Boccaccio.txt', 'Giuseppe Parini.txt', 'Italo Calvino.txt', 'Italo Svevo.txt', 'Ludovico Ariosto.txt', 'Luigi Pirandello.txt', 'Pier Paolo Pasolini.txt', 'Primo Levi.txt', 'Torquato Tasso.txt', 'Ugo Foscolo.txt', 'Umberto Eco.txt', 'Vittorio Alfieri.txt']


In [80]:
path_cade_autori = 'C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Preprocessing/autori/Corpus CADE/'

In [81]:
for file in files_autori:
    with open(path_autori + file, encoding='utf-8') as read_file:
        sentences = [simple_preproc(k).lower().split() for k in read_file.readlines()]
        with open(path_cade_autori+'CADE_'+file, mode='w', encoding='utf-8') as f:
            for s in sentences:
                for w in s:
                    f.write(w+" ")
                f.write('\n')